In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
# import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch import Tensor
from torch.nn.parameter import Parameter

import math
import warnings

# import torch
# from torch import Tensor
# from torch.nn.parameter import Parameter
# from torch.nn import init
# from torch.nn import Module
# # from torch.nn import _single, _pair, _triple, _reverse_repeat_tuple

# from torch.nn import common_types
# from torch.nn import _size_1_t, _size_2_t, _size_3_t
# from typing import Optional, List, Tuple

In [13]:
BW=6
BX=6
VBLMAX = 0.8 
T0 = 100e-12
kn = 220e-6
Vt = 0.4
alpha = 1.8
CBL = 270e-15
VWL = 0.9
Icell = kn*np.power(VWL-Vt,alpha) # Ideal cell current of the discharge path
delta_VBL_LSB = T0*Icell/CBL #The voltage difference on VBL created by the LSB
kclip = VBLMAX/delta_VBL_LSB 
#kclip = 10000+VBLMAX/delta_VBL_LSB
sigma_Vt = 23.8e-3
sigma_D = alpha*sigma_Vt/(VWL-Vt)

def quantizeWeight(W,BW):
    W = torch.min(W,(1.0-(2**(-(BW-1.0))))*torch.ones_like(W))
    Wbs = []
    Wbi = torch.lt(W,torch.zeros_like(W)).float()
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = torch.ge(W,0.5*torch.ones_like(W)).float()
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = torch.ge(W,0.5*torch.ones_like(W)).float()
    for i in range(BW):#-1):
        j = BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = torch.gt(Wbs[j],1.5*torch.ones_like(Wbs[j])).float()
        Wbs[j] = Wbs[j]*torch.ne(Wbs[j],2.0*torch.ones_like(Wbs[j]))
    return Wbs

def reconstructWeight(Wbs,BW):
    W = torch.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = (0.5)**j
        if (j == 0):
            multiplier = -1.0
        W += Wbs[j] * multiplier
    return W

class DIMAConv2d(nn.Conv2d):
    def __init__(
        self,
        sigma_D = 0,
        layer_index = 0,
        *kargs,
        **kwargs
    ):
#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
        super(DIMAConv2d, self).__init__(*kargs,**kwargs)
#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
#         self.output_h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
#         self.output_w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
        self.layer_index = layer_index
        self.noise = np.random.normal(0,sigma_D,(BW,self.weight.size()[0],self.weight.size()[1]
                                                 ,self.weight.size()[2],self.weight.size()[3]))
        
    def quantize_activations(self,input):
        if(self.layer_index != 0):
            input = torch.clamp(input,0,6) / 6
            input = 6 * torch.min(self.round_f(input*(2**BX))*(2**(-BX)) ,(1.0-(2**(-BX)))*torch.ones_like(input))
        else:
            input = torch.min(self.round_f(input*(2**BX))*(2**(-BX)) ,(1.0-(2**(-BX)))*torch.ones_like(input))
        return input
    
    def quantize_outputs(self,output):
        output = torch.clamp(output,-6,6)
        output = torch.min(self.round_f((output/6)*(2**(BW-1.0)))*(2.0**(1.0-BW)),(1.0-(2.0**(1.0-BW)))*torch.ones_like(output))
        output = output * 6
        return output 
               
    def round_f(self, x): #rounds a number to the nearest integer with STE for gradients
        x_r = torch.round(x)
        x_g = x
        return (x_r - x_g).detach() + x_g
    
    def quantize_weights(self):
        weight_q = quantizeWeight(self.weight.data,BW)
        for b in range(BW-1):
            weight_q[b+1] = weight_q[b+1]*(1+self.noise[b])
        weight = reconstructWeight(weight_q,BW)
        Wmax = kclip*np.power(2.0,-(BW-1))
        weight = torch.clamp(weight,-Wmax,Wmax)
        return (weight - self.weight).detach() + self.weight
        
    def _conv_forward(self, input, weight):
        if self.padding_mode != 'zeros':
#             print(1)
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, self.bias, self.stride,
                            _pair(0), self.dilation, self.groups)
#         print(2)
        return F.conv2d(input, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input: Tensor) -> Tensor:
        weight = self.quantize_weights()
        input = self.quantize_activations(input)
#         print(weight.size())
        output_h = int((((input.size()[2] + (2 * self.padding[0]) - 
                            ( self.dilation[0] * (self.kernel_size[0] - 1) ) - 1 )/ self.stride[0]) + 1) // 1)
        output_w = int((((input.size()[3] + (2 * self.padding[1]) - 
                            ( self.dilation[1] * (self.kernel_size[1] - 1) ) - 1 )/ self.stride[1]) + 1) // 1)
        
        if(self.kernel_size[0]*self.kernel_size[1]*self.in_channels > 256):
            print(1)
            weights = []
            inputs = []
            val = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
            coeff = self.in_channels // val
            
#             rem = self.in_channels % val
#             print(val, coeff)
            for i in range (val):
                if(i != val-1):
                    temp_weight = torch.zeros_like(weight)
                    temp_input = torch.zeros_like(input)
                    temp_weight[:,i*coeff:(i+1)*coeff,:,:] = weight[:,i*coeff:(i+1)*coeff,:,:]
                    weights.append(temp_weight)
#                     print(weight[:,i*coeff:(i+1)*coeff,:,:].size())
                    temp_input[:,i*coeff:(i+1)*coeff,:,:] = input[:,i*coeff:(i+1)*coeff,:,:]
                    inputs.append(temp_input)
#                     print(input[:,i*coeff:(i+1)*coeff,:,:].size())
                else:
#                     temp_weight = torch.zeros(weight.size()[0],coeff,weight.size()[2],weight.size()[3])
#                     temp_input = torch.zeros(input.size()[0],coeff,input.size()[2],input.size()[3])
#                     temp_input[:,:input.size()[1]-i*coeff,:,:] = input[:,i*coeff:,:,:]
#                     print(temp_weight.size())
#                     print(weight[:,i*coeff:,:,:].size())
                    temp_weight = torch.zeros_like(weight)
                    temp_input = torch.zeros_like(input)
                    temp_weight[:,i*coeff:,:,:] = weight[:,i*coeff:,:,:]
                    weights.append(temp_weight)
                    temp_input[:,i*coeff:,:,:] = input[:,i*coeff:,:,:]
                    inputs.append(temp_input)
            output = torch.zeros((input.size()[0],self.out_channels,output_h,output_w))
#             prin(output.size())
            for i in range (val):
#                 output = self._conv_forward(inputs[i],weights[i])
#                 print(out.size())
#                 output += self.quantize_outputs(out)
                if(i != 0):
                    output += self._conv_forward(inputs[i],weights[i])
                else:
                    output += self._conv_forward(inputs[i],weights[i])
            output = self.quantize_outputs(output)
        
#             output_ = self._conv_forward(input, weight)
#             print(torch.var(output), torch.var(output_))

#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
#         print(self.groups)
#         if(self.groups != 1):
#             weight = weight.reshape(self.groups,self.out_channels//self.groups, self.in_channels, weight.size()[2], weight.size()[3])
#         output =  self._conv_forward(input, weight)
#         output = self.quantize_outputs(output)
#         print(output.size())
#         if(self.groups != 1):
#             output = torch.sum(output, 2)
        else:
            output = self._conv_forward(input, weight)
            output = self.quantize_outputs(output)
        return output

In [14]:
class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter:
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = ProgressMeter._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print2(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    @classmethod
    def _get_batch_fmtstr(cls, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size=3, stride=1, padding=2), 
#             DIMAConv2d(in_channels = 3, out_channels = 64, kernel_size=3, stride=1, padding=2, sigma_D = sigma_D, layer_index = 0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
#             nn.Conv2d(in_channels = 64, out_channels = 192, kernel_size=3, padding=2),
            DIMAConv2d(in_channels = 64, out_channels = 192, kernel_size=3, padding=2,sigma_D = sigma_D, layer_index = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

        self.nclass = num_classes

    def forward(self, x):
        conv_features = self.features(x)
        flatten = conv_features.view(conv_features.size(0), -1)
        fc = self.fc_layers(flatten)
        return fc


def get_datasets(*args, **kwargs):
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]
    )

    trainset = torchvision.datasets.CIFAR10(train=True, transform=transform, *args, **kwargs)
    testset = torchvision.datasets.CIFAR10(train=False, transform=transform, *args, **kwargs)
    return trainset, testset


def get_dataloaders(trainset, testset, batch_size=100, num_worker=4):
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_worker)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_worker)

    return trainloader, testloader


def get_model(model_src_path, device='cpu'):
    model = Net(num_classes=10)
    state_dict = torch.load(model_src_path, map_location=device)
    model.load_state_dict(state_dict)
    return model


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        if type(output) is tuple:
            _, _, output = output
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res, pred[0, :]


def eval_single_batch_compute(x, y, model):
    output = model(x)
    accs, predictions = accuracy(output, y, topk=(1,))
    acc = accs[0]
    return acc, predictions


def eval_model(model, dataloader, print_acc=False, device='cpu', log_update_feq=20):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(
        len(dataloader),
        [top1],
        prefix='Evaluating Batch'
    )

    with torch.no_grad():
        for idx, data in enumerate(dataloader):
            x, y = data

            x = x.to(device)
            y = y.to(device)
            n_data = y.size(0)

            acc, predictions = eval_single_batch_compute(x, y, model)

            top1.update(acc.item(), n_data)
            if idx % log_update_feq == log_update_feq - 1:
                progress.print2(idx + 1)
#         print('asd')
        if print_acc:
            print(' * Acc@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg


def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('using device:',device)
    trainset, testset = get_datasets(root='./data', download=True)
    _, testloader = get_dataloaders(trainset, testset, batch_size=100, num_worker=16)

    model_src_path = '/Users/Alkin/Documents/ECE498/model.tar' # todo you need to set the path to downloaded model !!
    model = get_model(model_src_path, device)
    
    
    eval_model(model, testloader, print_acc=True, device=device)


# if __name__ == '__main__':
#     main()

In [15]:
if __name__ == '__main__':
    main()

using device: cpu
Files already downloaded and verified
Files already downloaded and verified


RuntimeError: Error(s) in loading state_dict for Net:
	Unexpected key(s) in state_dict: "features.3.bias". 